In [7]:
#
# Script to create a CSV file for the cats vs dogs dataset
#
import csv
import os

# Directory paths
cats_dir = "../data/train/cnn_train_images/cats"
dogs_dir = "../data/train/cnn_train_images/dogs"
csv_dir = '../data/train/cnn_train_images'

# File path within the folder
csv_file_path = os.path.join(csv_dir, "cats_vs_dogs_train.csv")

# List to store image paths and labels
data = []

# Process cats images
for filename in os.listdir(cats_dir):
    filepath = os.path.join(cats_dir, filename)
    
    # Splitting the string at the first occurrence of '.'
    id_number = filename.split('.', 1)

    # Keeping only the part before the first '.'
    id = id_number[0]

    data.append([id, 1, 0])  # Label 1 for cat

# Process dogs images
for filename in os.listdir(dogs_dir):
    filepath = os.path.join(dogs_dir, filename)
            
    # Splitting the string at the first occurrence of '.'
    id_number = filename.split('.', 1)

    # Keeping only the part before the first '.'
    id = id_number[0]

    data.append([id, 0, 1])  # Label 0 for non-hotdog

# Shuffle the data
import random
random.shuffle(data)

# Write data to CSV file
with open(csv_file_path, "w", newline="") as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(["Id", "Cat", "Dog"])
    csvwriter.writerows(data)


In [8]:
#
# Append labels from one CSV file to another (combine other train labels with cats vs dogs labels)
#

# Function to read CSV file and return a dictionary where keys are IDs
def read_csv_to_dict(file_path):
    data_dict = {}
    with open(file_path, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data_dict[row['Id']] = row
    return data_dict

# Paths to the CSV files
csv1_path = "../data/train/train.csv"
csv2_path = "../data/train/cats_vs_dogs_train.csv"

# Read CSV files into dictionaries
csv1_data = read_csv_to_dict(csv1_path)
csv2_data = read_csv_to_dict(csv2_path)

# Append labels from CSV1 to CSV2 if IDs match
for id, row in csv2_data.items():
    if id in csv1_data:
        csv2_data[id].update(csv1_data[id])

# Write combined data to a new CSV file
combined_csv_path = "../data/train/cats_vs_dogs_train.csv"
with open(combined_csv_path, "w", newline='') as csvfile:
    fieldnames = list(csv2_data.values())[0].keys()
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(csv2_data.values())


In [7]:
import torch
import os
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from PIL import Image

# Define relevant variables for the ML task
batch_size = 32
num_classes = 2 # 2 classes: Cat and Dog
learning_rate = 0.001
num_epochs = 30

# Step 1: Prepare the Dataset
class CustomDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir,
                                self.data.iloc[idx, 0]) + ".jpg"
        image = Image.open(img_name)
        label = self.data.iloc[idx, 1]

        if self.transform:
            image = self.transform(image)

        return image, label
    
# Define transformations
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load dataset
dataset = CustomDataset(csv_file='../data/train/cnn_train_images/cats_vs_dogs_train.csv',
                              root_dir='../data/train/cnn_train_images/',
                              transform=transform)

# Split dataset into training, validation, and test sets
train_size = int(0.6 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

# Define dataloaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [2]:
# Step 2: Define the CNN Architecture
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)
        self.fc1 = nn.Linear(128 * 6 * 6, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv3(x))
        x = torch.max_pool2d(x, 2)
        x = x.view(-1, 128 * 6 * 6)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [8]:
# Step 3: Train the Model

# WINDOWS: Device will determine whether to run the training on GPU or CPU.
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# MAC: Device will determine whether to run the training on GPU or CPU.
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')

model = CNN().to(device)
criterion = nn.CrossEntropyLoss()

# Set optimizer with optimizer
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9) 
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:    # Print every 100 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

print('Finished Training')


Epoch [1/50], Loss: 0.7133
Epoch [2/50], Loss: 0.7051
Epoch [3/50], Loss: 0.6218
Epoch [4/50], Loss: 0.5386
Epoch [5/50], Loss: 0.6632
Epoch [6/50], Loss: 0.3937
Epoch [7/50], Loss: 0.3775
Epoch [8/50], Loss: 0.2649
Epoch [9/50], Loss: 0.2576
Epoch [10/50], Loss: 0.1082
Epoch [11/50], Loss: 0.0733
Epoch [12/50], Loss: 0.0992
Epoch [13/50], Loss: 0.0326
Epoch [14/50], Loss: 0.0607
Epoch [15/50], Loss: 0.0164
Epoch [16/50], Loss: 0.0064
Epoch [17/50], Loss: 0.0035
Epoch [18/50], Loss: 0.0014
Epoch [19/50], Loss: 0.0008
Epoch [20/50], Loss: 0.0011
Epoch [21/50], Loss: 0.0008
Epoch [22/50], Loss: 0.0009
Epoch [23/50], Loss: 0.0004
Epoch [24/50], Loss: 0.0002
Epoch [25/50], Loss: 0.0002
Epoch [26/50], Loss: 0.0001
Epoch [27/50], Loss: 0.0002
Epoch [28/50], Loss: 0.0001
Epoch [29/50], Loss: 0.0001
Epoch [30/50], Loss: 0.0001
Epoch [31/50], Loss: 0.0001
Epoch [32/50], Loss: 0.0001
Epoch [33/50], Loss: 0.0001
Epoch [34/50], Loss: 0.0001
Epoch [35/50], Loss: 0.0001
Epoch [36/50], Loss: 0.0001
E

In [11]:
# Step 4: Evaluate the Model

# Validation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

val_accuracy = correct / total
print(f"Validation Accuracy: {val_accuracy:.2%}")

# Test
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = correct / total
print(f"Test Accuracy: {test_accuracy:.2%}")

Validation Accuracy: 35.00%
Test Accuracy: 50.00%


In [5]:
torch.save(model.state_dict(), '../data/train/cnn_train_images/model/cats_vs_dogs.pth')

In [6]:
import torch
import torchvision.transforms as transforms
from PIL import Image

# Step 1: Preprocess the Image
def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((64, 64)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
    image = Image.open(image_path)
    image = transform(image)
    return image.unsqueeze(0)  # Add batch dimension

# Step 2: Load the Model
model = CNN()  # Assuming you've defined your CNN model

# Load the trained model weights
model.load_state_dict(torch.load('../data/train/cnn_train_images/model/cats_vs_dogs.pth'))  # Replace 'model_weights.pth' with your model file path

# Step 3: Perform Inference
def predict_image(image_path, model):
    image = preprocess_image(image_path)
    model.eval()
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
        return predicted.item()

# Step 4: Post-process the Predictions
def get_class_label(class_index):
    if class_index == 0:
        return 'Cat'
    elif class_index == 1:
        return 'Dog'
    else:
        return 'Unknown'

# Example usage
image_path = '../data/train/cnn_train_images/b1a71c6f1943bd1e7c40f0ed09fb996a.jpg'  # Replace with the path to your image
predicted_class = predict_image(image_path, model)
predicted_label = get_class_label(predicted_class)
print('Predicted class:', predicted_label)


Predicted class: Cat
